In [1]:
import numpy as np
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import sklearn

In [2]:
df = pd.read_csv('NHIS_OPEN_GJ_2017_100.csv',engine = 'python')

In [3]:
df_sample = df[['총콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤', 'LDL콜레스테롤','성별코드', '연령대코드(5세단위)','수축기혈압','이완기혈압','식전혈당(공복혈당)','신장(5Cm단위)', '체중(5Kg단위)', '허리둘레']]


In [4]:
df_sample['총콜레스테롤']

0         193.0
1         228.0
2         136.0
3         201.0
4         199.0
          ...  
999995    198.0
999996    133.0
999997    205.0
999998    122.0
999999    225.0
Name: 총콜레스테롤, Length: 1000000, dtype: float64

In [5]:
df_sample = df_sample.dropna()

In [6]:
df_sample = df_sample.loc[(df_sample['허리둘레'] >= 40) & (df_sample['허리둘레'] <= 140)]

In [7]:
def bmi(height,weight):
    m = height / 100
    return weight / (m ** 2)
def diabetes(x):
    if x >= 100 and x < 126:
        return "1"
    elif x >= 127:
        return "2"
    else:
        return "0"

def fat(bmi) :
    if(bmi < 25):
        return '1'
    elif(bmi >= 25 and bmi < 30):
        return '2'
    else:
        return '3'

def pulse(low,high):
    if low >= 100 or high >= 160:
        return "3"
    elif 90 <= low < 100 or 140 <= high < 160:
        return "2"
    elif 80 <= low < 90 or 120 <= high < 140:
        return "1"
    else:
        return "0"

def bokbu(gen, hurry):
    if gen == 1 and hurry >= 90:
        return "1"
    elif gen == 2 and hurry >= 85:
        return "1"
    else:
        return "0"

def colesterol(all, ldl, hdl):
    if 139 <= all <200 or 130<= ldl <= 159 or 35 <= hdl <=  59:
        return 1
    elif all <= 200 or ldl <= 130 or hdl <= 60:
        return 0
    else:
        return -1

In [8]:
df_sample['당뇨병'] = df_sample['식전혈당(공복혈당)'].map(diabetes)
df_sample['고혈압'] = np.vectorize(pulse)(df_sample['이완기혈압'], df_sample['수축기혈압'])
df_sample['bmi'] = np.vectorize(bmi)(df_sample['신장(5Cm단위)'], df_sample['체중(5Kg단위)'])
df_sample['비만도'] = df_sample['bmi'].map(fat)
df_sample['복부비만'] = np.vectorize(bokbu)(df_sample['성별코드'], df_sample['허리둘레'])
df_sample['고지혈증'] = np.vectorize(colesterol)(df_sample['총콜레스테롤'],df_sample['LDL콜레스테롤'],df_sample['HDL콜레스테롤'])

In [9]:
df_x = df_sample[['고혈압','비만도','복부비만','고지혈증','HDL콜레스테롤',"LDL콜레스테롤","총콜레스테롤" ]]

In [10]:
df_x.describe()

,고지혈증,HDL콜레스테롤,LDL콜레스테롤,총콜레스테롤
count,996739.000000,996739.000000,996739.000000,996739.000000
mean,0.837462,56.930715,112.999331,195.501086
std,0.439787,17.229834,35.842479,38.669277
min,-1.000000,1.000000,1.000000,30.000000
25%,1.000000,46.000000,89.000000,169.000000
50%,1.000000,55.000000,111.000000,193.000000
75%,1.000000,66.000000,135.000000,219.000000
max,1.000000,8110.000000,5119.000000,2344.000000


In [11]:
## 디시젼트리

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x, df_sample["당뇨병"], test_size=0.2, random_state = 11)

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

df_clf = DecisionTreeClassifier(random_state=11)
df_clf.fit(X_train, y_train)
pred = df_clf.predict(X_test)
accuracy_score(y_test,pred)

0.541921664626683

In [14]:
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
X_train, X_test, y_train, y_test = train_test_split(df_x, df_sample["당뇨병"], test_size=0.2, random_state = 11)
rf_clf =RandomForestClassifier(random_state = 0)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy_score(y_test, pred)

0.5666071392740334

In [15]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_preds = lr_clf.predict(X_test)

accuracy_score(y_test, lr_preds)

0.628769789513815

In [16]:
lr_clf

LogisticRegression()